In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy import pi
from scipy.fftpack import fft
from scipy import stats
import pywt
import matplotlib.gridspec as gridspec
from itertools import combinations 
import sys

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import make_scorer
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler

In [ ]:
!pip install pandas_datareader
from pandas_datareader import data as pdr
from tqdm import tqdm,trange
stock_list = ['ADVANC','AEONTS','AMATA','AOT','AP','AWC','BANPU','BBL','BCH','BCP',
            'BCPG','BDMS','BEC','BEM','BGC','BGRIM','BH','BJC','BPP','BTS',
            'CBG','CENTEL','CHG','CK','CKP','COM7','CPALL','CPF','CPN',
            'DELTA','DTAC','EA','EGCO','EPG','ERW','ESSO','GFPT','GLOBAL','GPSC',
            'GULF','GUNKUL','HANA','HMPRO','INTUCH','IRPC','IVL','JAS','JMT','KBANK',
            'KCE','KKP','KTB','KTC','LH','MAJOR','MBK','MEGA','MINT','MTC',
            'ORI','OSP','PLANB','PRM','PSH','PSL','PTG','PTT','PTTEP','PTTGC',
            'QH','RATCH','RS','SAWAD','SCB','SCC','SGP','SPALI','SPRC','STA',
            'STEC','STPI','SUPER','TASCO','TCAP','THAI','THANI','THG','TISCO','TKN',
            'TMB','TOA','TOP','TPIPP','TQM','TRUE','TTW','TU','VGI','WHA' ]
stock_data = []
stock_name = []
for quote in tqdm(stock_list):
    try:
        data = pdr.get_data_yahoo(f'{quote}.BK', start='2016-01-01', end='2019-12-31')
        stock_data.append(data)
        stock_name.append(quote)
        #data.to_csv('/content/drive/My Drive/paper IS/code stock/{:}.csv'.format(quote))
    except:
        print("Error:", sys.exc_info()[0])
        print("Description:", sys.exc_info()[1])

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
100%|██████████| 99/99 [00:41<00:00,  2.40it/s]


In [ ]:
temp = []

for s_data in stock_data:
    #print(s_data)
    s_data['daily_return'] = s_data[['Close']].rolling(window=2).apply(lambda x:  x[1] / x[0] if x[0] != x[1] else 0 )
    s_data = s_data.iloc[1:, :]
    temp.append(s_data)

stock_data = temp

In [ ]:
mode_lenght = stats.mode([data.shape[0] for data in stock_data])[0][0]
stock_data_filtered = []
stock_list_filtered = [] 

for i in range(len(stock_list)): 
    if stock_data[i].shape[0] == mode_lenght:
        stock_data_filtered.append(stock_data[i])
        stock_list_filtered.append(stock_list[i])

In [ ]:
def reshape_two_dim(np_array): 
    return np_array.reshape([1, 1, np_array.shape[0]])

def create_wavelet_row(data, waveletname, level):
    result = []
    first_coeff = []

    for i in range(level):
        
        (data, coeff_d) = pywt.dwt(data, waveletname) 
        #reconstructed_signal = pywt.idwt(cA1, cD1, 'db2', 'smooth')
        
        if i == 0:
            first_coeff = list(coeff_d)
        result += list(data)

    result += first_coeff

    return result

In [ ]:
def create_wavelet(waveletname, level):
    # windows parameter
    max_data_lenght = max_lenght
    window_lenght = 90 
    window_preiod = 7
    window_ship = window_lenght - window_preiod

    # calculate dimension 
    temp = window_lenght
    first_dim = 0
    dim = 0
    for i in range(level): 
        temp = (temp+1)//2
        if i == 0: 
            first_dim = temp
        dim += temp 
    dim += first_dim 

    global stock_data_filtered
    windows_objs = []

    max_index = max_data_lenght // (window_ship) * (window_ship)
    # index_end = data_lenght 

    for index in range(0, max_index, window_ship): 
        index_start = - index - window_lenght
        index_end = - index

        wavelet_table = []

        for data_with_columns in stock_data_filtered: 
            if data_with_columns.shape[0] < -index_start: 
                continue
            data = data_with_columns['daily_return'][index_start:] if index == 0 else data_with_columns['daily_return'][index_start:index_end]
            wavelet_row = create_wavelet_row(data, waveletname, level) 
            wavelet_table += [wavelet_row]
            
        windows_obj = {} 

        for cal_score_obj in cal_scores:
            labels, score = k_mean(wavelet_table, cal_score_obj)

            windows_obj[cal_score_obj['name']] = { 
                'wavelet': wavelet_table, 
                'labels': labels,
                'score': score
            }

        windows_objs.append(windows_obj)

    # while index_start >= 0:
    #     wavelet_table = []

    #     for data_with_columns in stock_data_filtered: 
    #         data_len = data_with_columns.shape[0]
    #         if data_len < 
    #         data = data_with_columns['Adj Close'][index_start:index_end]
    #         wavelet_row = create_wavelet_row(data, waveletname, level) 
    #         wavelet_table += [wavelet_row]

    #     labels, score = k_mean(wavelet_table)

    #     windows_obj = { 
    #         'wavelet': wavelet_table, 
    #         'labels': labels,
    #         'score': score
    #     }

    #     windows_objs.append(windows_obj)

    #     # update index 
    #     index_start -= window_lenght - window_preiod
    #     index_end -= window_lenght - window_preiod
    
    windows_objs = windows_objs[::-1]

    wavelet_obj = {
        'windows': windows_objs,
    }
    for cal_score_obj in cal_scores: 
        score_name = cal_score_obj['name']
        wavelet_obj['avg_'+score_name] = sum([window[score_name]['score'] for window in windows_objs]) / len(windows_objs)

    return wavelet_obj

In [ ]:
cal_scores = [{
      'name': 'silhouette_score',
      'fn': silhouette_score,
      'optimize': max
  },{
      'name': 'davies_bouldin_score',
      'fn': davies_bouldin_score,
      'optimize': min
  },{
      'name': 'calinski_harabasz_score',
      'fn': calinski_harabasz_score,
      'optimize': max
  }]

def find_k(X, cal_score_obj):
    score_fn = cal_score_obj['fn']
    score_optimize = cal_score_obj['optimize']
    range_n_clusters = np.arange(3,20)
    scores = []
    cluster_center = {}
    for n_clusters in range_n_clusters:
        

        # def euc_dist(X, Y = None, Y_norm_squared = None, squared = False):
        #     return pairwise_distances(X, metric = 'cosine', n_jobs = -1)
        # #     #return np.arccos(cosine_similarity(X, Y))/np.pi

        # KMeans.euclidean_distances = euc_dist
        # #print('euclidean_distances')
        clusterer = KMeans(n_clusters=n_clusters, init='k-means++',max_iter=1000)
        clusterer = clusterer.fit(X)
        scores.append(score_fn(X, clusterer.predict(X)))
        cluster_center.update({'{}'.format(n_clusters):clusterer.cluster_centers_})
    return range_n_clusters[scores.index(score_optimize(scores))] 

def k_mean(X, cal_score_obj):
    global stock_list_filtered 
    score_fn = cal_score_obj['fn']
    X = StandardScaler().fit_transform(X)
    k = find_k(X, cal_score_obj)

    # def euc_dist(X, Y = None, Y_norm_squared = None, squared = False):
    #     return pairwise_distances(X, metric = 'correlation', n_jobs = -1)
    # #     #return np.arccos(cosine_similarity(X, Y))/np.pi

    # KMeans.euclidean_distances = euc_dist

    clusters = KMeans(n_clusters = k, init='k-means++',max_iter=1000)
    clusters.fit(X)

    labels = [ [] for i in range(k)]
    for i, label in enumerate(clusters.labels_): 
        labels[label] += [stock_list_filtered[i]]
    return labels, score_fn(X, clusters.labels_)

In [ ]:
list_wt = ['bior1.1', 'bior2.2', 'bior2.4', 
 'bior3.1', 'bior3.9', 'bior4.4', 'bior5.5', 
 'bior6.8', 'coif1', 'coif2', 'coif3', 'coif4',
 'coif9', 'coif10', 'coif15', 'coif16', 'coif17',
 'db1', 'db2', 'db3', 'db4', 
  'db26', 'db27', 'db28',  'db34', 'db35', 'db36', 'db37'
 , 'db38', 'dmey', 'haar', 'rbio1.1', 'rbio2.2'
 , 'rbio3.5', 'rbio3.7', 'rbio3.9', 'rbio4.4', 'rbio5.5', 'rbio6.8', 'sym2', 'sym3',
 'sym16', 'sym17', 'sym18', 'sym19', 'sym20']


In [ ]:
wavelet_parameter= ['haar', 'db1', 'db2', 'db3', 'db4','db26', 'db27', 'db28',  'db34', 'db35', 'db36', 'db37', 'db38']
max3rank = ['db38','coif17',  'dmey']
level = 2

wavelet_names = list_wt
wavelet_table = {}

for wavelet_name in wavelet_names: 
    wavelet_table[wavelet_name] = create_wavelet(wavelet_name, level)

In [ ]:
for wavelet_name in wavelet_names: 
    score_display = ''
    for cal_score_obj in cal_scores: 
        score_name = cal_score_obj['name']
        avg_score = wavelet_table[wavelet_name]['avg_' + score_name]
        score_display += '\t {:}: {:8.6f}'.format(score_name, avg_score)
    print('wavelet: {:7} {:}'.format(wavelet_name, score_display))

wavelet: bior1.1 	 silhouette_score: 0.040843	 davies_bouldin_score: 1.999535	 calinski_harabasz_score: 3.250712
wavelet: bior2.2 	 silhouette_score: 0.046083	 davies_bouldin_score: 1.979260	 calinski_harabasz_score: 2.828569
wavelet: bior2.4 	 silhouette_score: 0.037007	 davies_bouldin_score: 1.880005	 calinski_harabasz_score: 2.968414
wavelet: bior3.1 	 silhouette_score: 0.019370	 davies_bouldin_score: 2.016935	 calinski_harabasz_score: 2.753075
wavelet: bior3.9 	 silhouette_score: 0.027938	 davies_bouldin_score: 2.006092	 calinski_harabasz_score: 3.109314
wavelet: bior4.4 	 silhouette_score: 0.044743	 davies_bouldin_score: 1.927399	 calinski_harabasz_score: 3.629115
wavelet: bior5.5 	 silhouette_score: 0.045475	 davies_bouldin_score: 1.887704	 calinski_harabasz_score: 4.438686
wavelet: bior6.8 	 silhouette_score: 0.040143	 davies_bouldin_score: 1.966964	 calinski_harabasz_score: 3.753510
wavelet: coif1   	 silhouette_score: 0.045871	 davies_bouldin_score: 1.906575	 calinski_harabasz

In [ ]:
# # correlation
# wavelet: db38    	 silhouette_score: 0.718954	 davies_bouldin_score: 0.554812	 calinski_harabasz_score: 133.693315
# wavelet: coif17  	 silhouette_score: 0.721539	 davies_bouldin_score: 0.526273	 calinski_harabasz_score: 139.995372
# wavelet: dmey    	 silhouette_score: 0.719515	 davies_bouldin_score: 0.541662	 calinski_harabasz_score: 129.814934

# # cosine
# wavelet: db38    	 silhouette_score: 0.717627	 davies_bouldin_score: 0.535833	 calinski_harabasz_score: 133.702446
# wavelet: coif17  	 silhouette_score: 0.720327	 davies_bouldin_score: 0.520331	 calinski_harabasz_score: 140.441201
# wavelet: dmey    	 silhouette_score: 0.720794	 davies_bouldin_score: 0.539038	 calinski_harabasz_score: 130.122441

# # Eucidean
# wavelet: db38    	 silhouette_score: 0.718954	 davies_bouldin_score: 0.537615	 calinski_harabasz_score: 133.701637
# wavelet: coif17  	 silhouette_score: 0.722503	 davies_bouldin_score: 0.519529	 calinski_harabasz_score: 140.691314
# wavelet: dmey    	 silhouette_score: 0.720588	 davies_bouldin_score: 0.541662	 calinski_harabasz_score: 130.049676

In [ ]:
# for cal_score_obj in cal_scores:
#     score_name = cal_score_obj['name']
#     print('='*80)
#     print(score_name)
#     print('='*80)
#     for wavelet_name in wavelet_names:  
#         print('-'*50)
#         print(wavelet_name)
#         print('-'*50)
#         for window in wavelet_table[wavelet_name]['windows']: 
#             print(window[score_name]['score'])

In [ ]:
for cal_score_obj in cal_scores:
    score_name = cal_score_obj['name']
    print('*'*50)
    print(score_name)
    for wavelet_name in wavelet_names:
        windows = wavelet_table[wavelet_name]['windows']
        avg_score = wavelet_table[wavelet_name]['avg_'+score_name] 
        print('='*50)
        print('wavelet: {:10} {:}: {:}'.format(wavelet_name, score_name, avg_score))
        for i, window in enumerate(windows): 
            window = window[score_name]
            print('-'*50)
            print('window {:}:'.format(i+1))
            for j, label in enumerate(window['labels']): 
                print('custer {:} have {:} stocks : {:}'.format(j+1, len(label), label))
            print('score of window: {:}.'.format(window['score']))
        print('\n\n\n')

Streaming output truncated to the last 5000 lines.
custer 10 have 2 stocks : ['QH', 'SUPER']
custer 11 have 5 stocks : ['ERW', 'GUNKUL', 'RATCH', 'STEC', 'THAI']
custer 12 have 1 stocks : ['SPRC']
custer 13 have 1 stocks : ['AOT']
custer 14 have 3 stocks : ['CENTEL', 'CHG', 'THANI']
custer 15 have 7 stocks : ['BCH', 'CK', 'PTG', 'PTT', 'SCC', 'TRUE', 'VGI']
custer 16 have 1 stocks : ['BEM']
custer 17 have 15 stocks : ['BANPU', 'BH', 'CPALL', 'ESSO', 'HMPRO', 'JMT', 'KCE', 'KTC', 'MBK', 'MEGA', 'PSL', 'PTTGC', 'RS', 'STA', 'TASCO']
custer 18 have 1 stocks : ['BDMS']
score of window: 1.815020570295294.
--------------------------------------------------
window 11:
custer 1 have 5 stocks : ['ADVANC', 'BEM', 'HMPRO', 'JMT', 'QH']
custer 2 have 13 stocks : ['AOT', 'BANPU', 'BCP', 'CK', 'DTAC', 'KBANK', 'PTTEP', 'RATCH', 'SCC', 'TCAP', 'TRUE', 'TU', 'VGI']
custer 3 have 5 stocks : ['KKP', 'ORI', 'PTTGC', 'STPI', 'THAI']
custer 4 have 6 stocks : ['BEC', 'CHG', 'IRPC', 'PTG', 'SGP', 'TMB']
cust

In [ ]:
rank_table = {}

for wavelet_name in wavelet_names: 
    rank_table[wavelet_name] = []

for cal_score_obj in cal_scores: 
    score_name = cal_score_obj['name'] 
    score_optimize = cal_score_obj['optimize']
    avg_scores = [(wavelet_name, wavelet_table[wavelet_name]['avg_' + score_name]) for wavelet_name in wavelet_names] 
    sorted_wavelet = [score_tuple[0] for score_tuple in sorted(avg_scores, key=lambda x:x[1], reverse=score_optimize==max)] 
    for index, wavelet_name in enumerate(sorted_wavelet):
        rank_table[wavelet_name].append(index) 

rank_list = [(wavelet_name, sum(rank_table[wavelet_name])) for wavelet_name in rank_table.keys()]
top_wavelet = sorted(rank_list, key=lambda x:x[1])[0][0]

In [ ]:
top_wavelet

'rbio3.7'

In [ ]:
new_matrix = {}
n_windows = len(wavelet_table[wavelet_names[0]]['windows'] )
n_stocks = len(stock_list_filtered)

for cal_score_obj_1 in cal_scores:
    score_name_1 = cal_score_obj_1['name']
    new_matrix[score_name_1] = {}
    for cal_score_obj_2 in cal_scores:
        score_name_2 = cal_score_obj_2['name']
        new_matrix[score_name_1][score_name_2] = np.full((n_stocks, n_stocks), 0, dtype=float)
        np.fill_diagonal(new_matrix[score_name_1][score_name_2], 1)

for cal_score_obj_1 in cal_scores:
    score_name_1 = cal_score_obj_1['name']
    for cal_score_obj_2 in cal_scores:
        score_name_2 = cal_score_obj_2['name']
        windows = wavelet_table[top_wavelet]['windows']
        for window in windows: 
            window = window[score_name_1] 
            for label in window['labels']: 
                for pair in combinations(label, 2):
                    index_i = stock_list_filtered.index(pair[0])
                    index_j = stock_list_filtered.index(pair[1]) 
                    new_matrix[score_name_1][score_name_2][index_i][index_j] += 1 / n_windows
                    new_matrix[score_name_1][score_name_2][index_j][index_i] += 1 / n_windows

In [ ]:
all_windows = {}

for cal_score_obj_1 in cal_scores:
    score_name_1 = cal_score_obj_1['name']
    all_windows[score_name_1] = {}
    for cal_score_obj_2 in cal_scores:
        score_name_2 = cal_score_obj_2['name']
        labels, score = k_mean(new_matrix[score_name_1][score_name_2], cal_score_obj_2)
        all_windows[score_name_1][score_name_2] = {
            'labels': labels,
            'score': score
        }

In [ ]:
print('wavelet: {:}.\n\n'.format(top_wavelet))

for cal_score_obj_1 in cal_scores:
    score_name_1 = cal_score_obj_1['name']
    print('='*50)
    print(score_name_1)
    for cal_score_obj_2 in cal_scores:
        score_name_2 = cal_score_obj_2['name']
        all_window = all_windows[score_name_1][score_name_2]
        print('-'*50)
        print('score method: {:10}, score: {:}'.format(score_name_2, all_window['score']))
    print('\n')

wavelet: rbio3.7.


silhouette_score
--------------------------------------------------
score method: silhouette_score, score: 0.19309050316931406
--------------------------------------------------
score method: davies_bouldin_score, score: 1.3384981732838395
--------------------------------------------------
score method: calinski_harabasz_score, score: 20.539115167256348


davies_bouldin_score
--------------------------------------------------
score method: silhouette_score, score: 0.07884168664647657
--------------------------------------------------
score method: davies_bouldin_score, score: 1.9392318079751794
--------------------------------------------------
score method: calinski_harabasz_score, score: 8.992466994806191


calinski_harabasz_score
--------------------------------------------------
score method: silhouette_score, score: 0.1583507141462027
--------------------------------------------------
score method: davies_bouldin_score, score: 1.2901333668687178
---------------

In [ ]:
print('wavelet: {:}.\n\n'.format(top_wavelet))

for cal_score_obj_1 in cal_scores:
    score_name_1 = cal_score_obj_1['name']
    print('='*50)
    print(score_name_1)
    for cal_score_obj_2 in cal_scores:
        score_name_2 = cal_score_obj_2['name']
        all_window = all_windows[score_name_1][score_name_2]
        print('-'*50)
        print('score method: {:10}, score: {:}'.format(score_name_2, all_window['score']))
        for i, label in enumerate(all_window['labels']): 
            print('custer {:} have {:} stocks : {:}'.format(i+1, len(label), label))
    print('\n')

wavelet: rbio3.7.


silhouette_score
--------------------------------------------------
score method: silhouette_score, score: 0.19309050316931406
custer 1 have 4 stocks : ['BBL', 'ESSO', 'SCC', 'SPALI']
custer 2 have 5 stocks : ['AP', 'COM7', 'GFPT', 'MEGA', 'TCAP']
custer 3 have 4 stocks : ['CHG', 'JAS', 'KTB', 'SUPER']
custer 4 have 8 stocks : ['CPALL', 'EGCO', 'GPSC', 'HANA', 'MTC', 'RS', 'THANI', 'WHA']
custer 5 have 8 stocks : ['ADVANC', 'BEC', 'BH', 'CBG', 'KCE', 'ORI', 'PTTGC', 'SAWAD']
custer 6 have 5 stocks : ['BEM', 'BTS', 'LH', 'QH', 'TTW']
custer 7 have 3 stocks : ['EA', 'MBK', 'PLANB']
custer 8 have 3 stocks : ['BANPU', 'HMPRO', 'VGI']
custer 9 have 9 stocks : ['AEONTS', 'AMATA', 'AOT', 'CPN', 'PSL', 'SCB', 'SGP', 'STPI', 'TKN']
custer 10 have 4 stocks : ['DTAC', 'EPG', 'JMT', 'STA']
custer 11 have 4 stocks : ['CK', 'CKP', 'CPF', 'MINT']
custer 12 have 2 stocks : ['MAJOR', 'THAI']
custer 13 have 6 stocks : ['BCH', 'KBANK', 'KTC', 'PTT', 'PTTEP', 'TASCO']
custer 14 have 6 